<a href="https://colab.research.google.com/github/MarceloMdeO/Data-Science-Projects/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Aula_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [6]:
import pandas as pd
import numpy as np
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GeminiAPIKey')
genai.configure(api_key = GOOGLE_API_KEY)

In [27]:
for tmoldels in genai.list_models():
  if 'embedContent' in tmoldels.supported_generation_methods:
    print(tmoldels.name)

models/embedding-001
models/text-embedding-004


In [12]:
title = "A próxima geração de IA para desenvolvedores do Google Workspace"
sample_text = ("Título a próxima geração de IA para desenvolvedores do Google Workspace"
"\n"
"Artigo completo: \n"
"\n"
"Gemini API e Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de AI generativa")

embeddings = genai.embed_content(model = 'models/embedding-001', content = sample_text, title = title, task_type = "RETRIEVAL_DOCUMENT")


print(embeddings)

{'embedding': [0.045686673, -0.038465124, -0.015621951, 0.014926582, 0.069403656, -0.003310409, -0.02311954, -0.01996707, 0.059901554, 0.058885198, 0.014630187, 0.017215187, -0.05115219, -0.023100747, 0.009156631, -0.032968953, 0.014100512, -0.007812754, -0.039236315, 0.0007077328, 0.012058299, 0.003687195, -0.0255381, -0.06667823, -0.022110978, 0.0138220135, 0.01270127, -0.03472386, -0.031644166, 0.01961573, -0.050903153, 0.044201605, -0.03648925, 0.012001866, -0.03717891, -0.040805113, -0.018880961, -0.04338396, -0.004624057, -0.005620403, 0.0048294333, -0.083439544, -0.010292581, 0.030273004, -0.0023022385, -0.02328816, 0.030849034, 0.04419913, 0.014411209, -0.068756856, 0.028308112, 0.015438934, 0.06704708, -0.038862795, -0.007491506, -0.015759027, 0.02185673, -0.033278752, 0.023571555, 0.00409605, -0.0010338553, 0.022274537, -0.023083903, 0.045267925, 0.018956922, -0.06038042, -0.04613022, -0.002291199, 0.011127788, 0.052052736, 0.015302941, 0.007710822, 0.06524822, -0.063399635, 

In [13]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}

DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}

DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [24]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df.head(5)

,Titulo,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [38]:
def emb (titulo, texto):
  return genai.embed_content(model = model, content = texto , title = titulo, task_type = "RETRIEVAL_DOCUMENT")["embedding"]

In [39]:
 model = "models/embedding-001"
 df["Embeddings"] = df.apply(lambda linha: emb(linha['Titulo'], linha['Conteudo']), axis = 1)
 df

,Titulo,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [48]:
def busca_consulta (consulta, base, modelo):
  embedding_consulta = genai.embed_content(model = modelo, content = consulta, task_type = 'retrieval_query')

  produto_escalar = np.dot(np.stack(base["Embeddings"]), embedding_consulta["embedding"])
  idx = np.argmax(produto_escalar)
  return base.iloc[idx]['Conteudo']

In [49]:
consulta = "Como faço para torcar marcha no carro do google"

trecho = busca_consulta(consulta, df, model)
print(trecho)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.
